In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/My\ Drive

 20ep_400bs_norm_sig.h5  'Colab Notebooks'   faces		    model.h5
 50ep_1000bs.h5		  emotions.zip	     list_bbox_celeba.csv


In [ ]:
!unzip -q "/content/drive/My Drive/emotions.zip"

In [ ]:
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv2D, Dropout, Dense, Flatten, MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

In [ ]:
TRAIN_IMG_DIR = '/content/emotions/train'
TRAIN_DS_SIZE = 28709

TEST_IMG_DIR = '/content/emotions/test'
TEST_DS_SIZE = 7178

BATCH_SIZE = 500

In [ ]:
train_idg = ImageDataGenerator(rescale=1/255)
test_idg = ImageDataGenerator(rescale=1/255)

In [ ]:
train_gen = train_idg.flow_from_directory(
    TRAIN_IMG_DIR,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE
)

test_gen = test_idg.flow_from_directory(
    TEST_IMG_DIR,
    target_size=(48, 48),
    color_mode="grayscale",
    class_mode="categorical",
    batch_size=BATCH_SIZE
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
regul = l2(0.001)

In [ ]:
model = Sequential([
    Conv2D(64, 3, input_shape=(48, 48, 1), padding='same', kernel_regularizer=regul),
    Activation('relu'),
    Conv2D(64, 3, padding='same' kernel_regularizer=regul),
    Activation('relu'),
    Dropout(0.5),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, 3, padding='same', kernel_regularizer=regul),
    Activation('relu'),
    Conv2D(128, 3, padding='same' kernel_regularizer=regul),
    Activation('relu'),
    Dropout(0.5),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, 3, padding='same', kernel_regularizer=regul),
    Activation('relu'),
    Conv2D(256, 3, padding='same' kernel_regularizer=regul),
    Activation('relu'),
    Dropout(0.5),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(512, 3, padding='same', kernel_regularizer=regul),
    Activation('relu'),
    Conv2D(512, 3, padding='same' kernel_regularizer=regul),
    Activation('relu'),
    Dropout(0.5),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu', kernel_regularizer=regul),
    Dense(1024, activation='relu', kernel_regularizer=regul),
    Dense(7, activation='softmax'),
])

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=1e-3, decay=1e-6), metrics=['accuracy', 'mse'])

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=TRAIN_DS_SIZE // BATCH_SIZE,
    epochs=100,
    validation_data=test_gen,
    validation_steps=TEST_DS_SIZE // BATCH_SIZE
)

Epoch 1/100
57/57 [==============================] - 13s 223ms/step - loss: 4.5465 - accuracy: 0.2668 - mse: 0.1176 - val_loss: 3.9180 - val_accuracy: 0.2471 - val_mse: 0.1191
Epoch 2/100
57/57 [==============================] - 12s 217ms/step - loss: 3.1642 - accuracy: 0.3866 - mse: 0.1021 - val_loss: 3.1411 - val_accuracy: 0.2474 - val_mse: 0.1193
Epoch 3/100
57/57 [==============================] - 12s 216ms/step - loss: 2.3792 - accuracy: 0.4946 - mse: 0.0887 - val_loss: 2.7540 - val_accuracy: 0.2600 - val_mse: 0.1189
Epoch 4/100
57/57 [==============================] - 12s 216ms/step - loss: 1.9460 - accuracy: 0.5606 - mse: 0.0802 - val_loss: 2.5117 - val_accuracy: 0.2870 - val_mse: 0.1175
Epoch 5/100
57/57 [==============================] - 12s 215ms/step - loss: 1.6984 - accuracy: 0.5904 - mse: 0.0756 - val_loss: 2.4675 - val_accuracy: 0.1807 - val_mse: 0.1219
Epoch 6/100
57/57 [==============================] - 12s 216ms/step - loss: 1.5504 - accuracy: 0.6128 - mse: 0.0724 - va

In [ ]:
save_model(model, '/content/100ep_500bs_65.h5')

In [ ]:
import pickle

with open('/content/100ep_500bs_65_h', 'wb') as write_file:
  pickle.dump(history.history, write_file)